In [ ]:
# 패키지 설치: yfinance, getFamaFrenchFactors
!pip install yfinance --quiet
!pip install getFamaFrenchFactors --quiet

In [ ]:
# 패키지 임포트: numpy, pandas, yfinance, matplotlib, seaborn, statsmodels, getFamaFrenchFactors, sklearn
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import getFamaFrenchFactors as gff
from sklearn.linear_model import Ridge, Lasso
sns.set()

In [ ]:
# 주가 데이터 다운로드
# 애플 주식 티커
ticker = 'AAPL'

# 시작일, 종료일
start = '2001-01-01'
end = '2022-10-12'

# 주가 데이터 다운로드
stock_data = yf.download(ticker, start, end, adjusted=True)

# 주가 수익률 계산
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = 'Month_Rtn'
stock_returns.index = pd.to_datetime(stock_returns.index).date

In [ ]:
# 파마-프렌치 3 팩터 데이터 다운로드
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={'date_ff_factors': 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)
ff3_monthly.index = pd.to_datetime(ff3_monthly.index).date

# 주가 데이터와 팩터 데이터 병합
ff_data = pd.concat([ff3_monthly, stock_returns], axis=1, join='inner')

In [ ]:
# ff_data 데이터프레임 칼럼 이름 변경
ff_data.columns = ['beta', 'size', 'value', 'rf', 'ret']

# 결과
ff_data

In [ ]:
# 회귀분석을 위한 설명변수와 종속변수 설정
# 설명변수: 팩터 데이터
# 종속변수: 주가 초과수익률
X = ff_data[['beta', 'size', 'value']]
y = ff_data['ret'] - ff_data['rf']

# RIdge 회귀분석
ridge = Ridge(alpha=0.1)
ridge.fit(X, y)
print(ridge.coef_, ridge.intercept_)

# Lasso 회귀분석
lasso = Lasso(alpha=0.001)
lasso.fit(X, y)
print(lasso.coef_, lasso.intercept_)

In [ ]:
# Ridge 회귀분석 계수 저장을 위한 빈 리스트
ridge_coef_list = []

# 패러미터의 범위 설정
alpha_arr = np.linspace(0.0, 4.0, 100)

# 패러미터 변화에 따른 Ridge 회귀분석
for alpha in alpha_arr:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X, y)
    ridge_coef_list.append(ridge.coef_)

In [ ]:
# 회귀분석 계수 데이터프레임
ridge_df = pd.DataFrame(ridge_coef_list, index=alpha_arr, columns=['Market', 'Size', 'Value'])

# 분석 결과 시각화
ridge_df.plot()
plt.title('Impact of Ridge Regularization')
plt.xlabel('Penalty')
plt.ylabel('Coefficient')

In [ ]:
# Lasso 회귀분석 계수 저장을 위한 빈 리스트
lasso_coef_list = []

# 패러미터의 범위 설정
alpha_arr = np.linspace(0.000, 0.010, 100)

# 패러미터 변화에 따른 Lasso 회귀분석
for alpha in alpha_arr:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    lasso_coef_list.append(lasso.coef_)

In [ ]:
# 회귀분석 계수 데이터프레임
lasso_df = pd.DataFrame(lasso_coef_list, index=alpha_arr, columns=['Market', 'Size', 'Value'])

# 분석 결과 시각화
lasso_df.plot()
plt.title('Impact of Lasso Regularization')
plt.xlabel('Penalty')
plt.ylabel('Coefficient')

Copyright 2022. 퀀트대디. All rights reserved.